In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression, load_diabetes, fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
print("🚀 Linear Regression Optimizer Comparison - FIXED")

# CHOOSE DATASET HERE (change this option)
DATASET_CHOICE = 0  # 0=Synthetic, 1=Diabetes, 2=California Housing, 3=make_regression

if DATASET_CHOICE == 0:
    # Synthetic data (2D for visualization)
    n_samples, n_features = 200, 2
    noise_level = 0.1
    np.random.seed(42)
    X = np.random.randn(n_samples, n_features)
    true_coef = np.array([2.0, -1.5])
    y = X @ true_coef + noise_level * np.random.randn(n_samples)
    print("📊 Synthetic  200 samples, 2 features")

elif DATASET_CHOICE == 1:
    # Diabetes dataset (real medical data)
    data = load_diabetes()
    X, y = data.data, data.target
    print(f"📊 Diabetes dataset: {X.shape[0]} samples, {X.shape[1]} features")

elif DATASET_CHOICE == 2:
    # California Housing (modern replacement for Boston)
    data = fetch_california_housing()
    X, y = data.data, data.target
    print(f"📊 California Housing: {X.shape[0]} samples, {X.shape[1]} features")

elif DATASET_CHOICE == 3:
    # Synthetic regression (scikit-learn generator)s
    X, y = make_regression(n_samples=500, n_features=10, noise=0.1, random_state=42)
    print("📊 Synthetic regression: 500 samples, 10 features")

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(f"✅ Train: {X_train.shape}, Test: {X_test.shape}")
print(f"📈 Data ready for optimization!")

In [ ]:
def optimize_all(X_train, y_train,
                 MAX_ITER_ADAGRAD=10000, LR_ADAGRAD=0.03, EARLY_STOP_ADAGRAD=True,  # MAX 10K!
                 MAX_ITER_NEWTON=20, EARLY_STOP_NEWTON=True):
    n, p = X_train.shape
    Xb = np.c_[np.ones((n, 1)), X_train]

    # 1. NORMAL EQUATION
    theta_normal = np.linalg.pinv(Xb.T @ Xb) @ Xb.T @ y_train
    normal_loss = 0.5 * np.mean((Xb @ theta_normal - y_train)**2)

    # 2. ADAGRAD (MAX 10K!)
    print(f"🚀 AdaGrad: MAX={MAX_ITER_ADAGRAD} iter, LR={LR_ADAGRAD}")
    theta_adagrad = np.zeros(Xb.shape[1])
    g2_acc = np.zeros_like(theta_adagrad) * 1e-6
    adagrad_losses = [float('inf')]
    adagrad_traj = [theta_adagrad.copy()]
    adagrad_iters_used = 0

    for i in range(MAX_ITER_ADAGRAD):  # Up to 10K!
        grad = Xb.T @ (Xb @ theta_adagrad - y_train) / n
        g2_acc += grad**2
        theta_adagrad -= LR_ADAGRAD * grad / (np.sqrt(g2_acc) + 1e-8)
        loss = 0.5 * np.mean((Xb @ theta_adagrad - y_train)**2)
        adagrad_losses.append(loss)
        adagrad_traj.append(theta_adagrad.copy())
        adagrad_iters_used = i + 1

        if EARLY_STOP_ADAGRAD and i > 100 and abs(adagrad_losses[-1] - adagrad_losses[-2]) < 1e-7:
            print(f"   ℹ️ AdaGrad STOPPED at {i+1}/{MAX_ITER_ADAGRAD}")
            break

    adagrad_traj = np.array(adagrad_traj)
    adagrad_losses = np.array(adagrad_losses)

    # 3. NEWTON
    print(f"🚀 Newton: MAX={MAX_ITER_NEWTON} iter")
    theta_newton = np.zeros(Xb.shape[1])
    newton_losses = [float('inf')]
    newton_traj = [theta_newton.copy()]
    newton_iters_used = 0

    for i in range(MAX_ITER_NEWTON):
        residuals = Xb @ theta_newton - y_train
        grad = Xb.T @ residuals / n
        hessian = Xb.T @ Xb / n
        delta = np.linalg.pinv(hessian) @ grad
        theta_newton -= delta
        loss = 0.5 * np.mean(residuals**2)
        newton_losses.append(loss)
        newton_traj.append(theta_newton.copy())
        newton_iters_used = i + 1

        if EARLY_STOP_NEWTON and loss < 1e-8:
            print(f"   ℹ️ Newton STOPPED at {i+1}/{MAX_ITER_NEWTON}")
            break

    newton_traj = np.array(newton_traj)
    newton_losses = np.array(newton_losses)

    return {
        'normal': theta_normal, 'normal_loss': normal_loss, 'normal_iters': 1,
        'adagrad': theta_adagrad, 'adagrad_losses': adagrad_losses,
        'adagrad_traj': adagrad_traj, 'adagrad_iters': adagrad_iters_used,
        'newton': theta_newton, 'newton_losses': newton_losses,
        'newton_traj': newton_traj, 'newton_iters': newton_iters_used
    }

# 🎛️ MAX 10K CONTROL PANEL
ADAGRAD_CONFIG = {
    'max_iter': 10000,    # MAX 10K! (change: 100, 500, 1000, 2500, 10000)
    'lr': 0.03,
    'early_stop': True
}

NEWTON_CONFIG = {
    'max_iter': 20,       # (change: 3, 5, 10, 20)
    'early_stop': True
}

# RUN
results = optimize_all(X_train, y_train,
                      MAX_ITER_ADAGRAD=ADAGRAD_CONFIG['max_iter'],
                      LR_ADAGRAD=ADAGRAD_CONFIG['lr'],
                      EARLY_STOP_ADAGRAD=ADAGRAD_CONFIG['early_stop'],
                      MAX_ITER_NEWTON=NEWTON_CONFIG['max_iter'],
                      EARLY_STOP_NEWTON=NEWTON_CONFIG['early_stop'])

print(f"\n✅ MAX 10K VERIFIED:")
print(f"   AdaGrad: {results['adagrad_iters']}/{ADAGRAD_CONFIG['max_iter']} iterations")
print(f"   Newton:  {results['newton_iters']}/{NEWTON_CONFIG['max_iter']} iterations")


In [ ]:
# Test MSE
Xb_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]
mse_normal = mean_squared_error(y_test, Xb_test @ results['normal'])
mse_adagrad = mean_squared_error(y_test, Xb_test @ results['adagrad'])
mse_newton = mean_squared_error(y_test, Xb_test @ results['newton'])

# FULL COMPARISON TABLE (Iterations REQUIRED, not max!)
comparison_df = pd.DataFrame({
    'Method': ['Normal Equation', 'AdaGrad', 'Newton Method'],
    'Final Train Loss': [results['normal_loss'],
                        results['adagrad_losses'][-1],
                        results['newton_losses'][-1]],
    'Test MSE': [mse_normal, mse_adagrad, mse_newton],
    'Iterations Required': [results['normal_iters'],
                          results['adagrad_iters'],
                          results['newton_iters']],  # ← ACTUAL iterations used!
})

print("\n📊 COMPLETE COMPARISON TABLE")
print("="*80)
print(comparison_df.round(6).to_string(index=False))
print(f"\n🏆 BEST METHOD: {comparison_df.loc[comparison_df['Test MSE'].idxmin(), 'Method']}")
print(f"📈 Best Test MSE: {comparison_df['Test MSE'].min():.6f}")

In [ ]:
# 1. MSE BAR CHART
plt.figure(figsize=(10, 6))
methods = ['Normal Eq.', 'AdaGrad', 'Newton']
mses = [mse_normal, mse_adagrad, mse_newton]
colors = ['#1f77b4', '#2ca02c', '#ff7f0e']
bars = plt.bar(methods, mses, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
plt.ylabel('Test MSE', fontsize=14, fontweight='bold')
plt.title('Test Performance Comparison', fontsize=16, fontweight='bold', pad=20)
plt.ylim(0, max(mses)*1.1)

for bar, mse in zip(bars, mses):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height*1.02, f'{mse:.4f}',
             ha='center', va='bottom', fontweight='bold', fontsize=12)
plt.tight_layout()
plt.show()
print("📊 1/2: Test MSE Comparison ✓")

# 2. CONVERGENCE CURVES (log scale)
plt.figure(figsize=(10, 6))
plt.semilogy(results['adagrad_losses'][1:], 'g-', lw=4, marker='o', label='AdaGrad', markersize=6)
plt.semilogy(results['newton_losses'][1:], 'orange', lw=4, marker='s', label='Newton', markersize=6)
plt.axhline(results['normal_loss'], color='blue', ls='--', lw=3,
           label=f'Normal Eq. (loss={results["normal_loss"]:.2e})')
plt.xlabel('Iteration', fontweight='bold')
plt.ylabel('Loss (log scale)', fontweight='bold')
plt.title('Convergence Speed Comparison', fontweight='bold', pad=20)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
print("📈 2/2: Convergence Curves ✓")


print("🌊 3/3: Loss Contours ✓")
print("✨ All visualizations complete!!")


In [ ]:
from IPython.display import HTML, display
import matplotlib.animation as animation
import matplotlib

# 🔧 FIX SIZE LIMIT
matplotlib.rcParams['animation.embed_limit'] = 128  # 128MB

print("🎬 Cell 5: 3D + 2D Animations (200 FRAMES - 10K Progress!)")

if results['adagrad_traj'].shape[1] < 3:
    print("⚠️ Run Cell 1 with DATASET_CHOICE=0!")
else:
    # Loss surface
    w0_range = np.linspace(-3, 3, 50)
    w1_range = np.linspace(-3, 3, 50)
    T0, T1 = np.meshgrid(w0_range, w1_range)
    L_surf = 0.5 * (T0**2 + 4*T1**2)
    true_theta = np.array([0.0, 0.0])

    colors = {
        'AdaGrad': '#2ca02c', 'Newton': '#ff7f0e', 'Normal': '#1f77b4'
    }

    def animate_convergence(view='3D'):
        fig = plt.figure(figsize=(12, 9))

        if view == '3D':
            ax = fig.add_subplot(111, projection='3d')
            surf = ax.plot_surface(T0, T1, L_surf, cmap='viridis', alpha=0.7)
            ax.set_zlabel('Loss', fontweight='bold')
            ax.scatter(true_theta[0], true_theta[1], 0, color='black', s=200, marker='*', label='True Min')
        else:
            ax = fig.add_subplot(111)
            cs = ax.contourf(T0, T1, L_surf, levels=30, cmap='viridis', alpha=0.8)
            ax.contour(T0, T1, L_surf, levels=10, colors='white', alpha=0.7, linewidths=0.8)
            ax.scatter(true_theta[0], true_theta[1], color='black', s=200, marker='*', label='True Min')
            plt.colorbar(cs, ax=ax, label='Loss', shrink=0.8)

        ax.set_xlabel('θ₀ (bias)', fontweight='bold')
        ax.set_ylabel('θ₁ (w1)', fontweight='bold')
        ax.set_title(f'Optimizer Paths - {len(results["adagrad_traj"])-1} Total Iterations ({view})',
                    fontsize=14, fontweight='bold')

        # Text: 3D vs 2D
        if view == '3D':
            iter_text = ax.text2D(0.02, 0.98, "", transform=ax.transAxes,
                                 fontsize=14, color='darkred', fontweight='bold',
                                 bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.9))
        else:
            iter_text = ax.text(0.02, 0.98, "", transform=ax.transAxes,
                               fontsize=14, color='darkred', fontweight='bold',
                               bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.9))

        # Lines + points
        lines, points = {}, {}
        for name, color in colors.items():
            if view == '3D':
                l, = ax.plot([], [], [], color=color, lw=3, label=name)
                p, = ax.plot([], [], [], color=color, marker='o', ms=8)
            else:
                l, = ax.plot([], [], color=color, lw=3, label=name)
                p, = ax.plot([], [], color=color, marker='o', ms=8)
            lines[name], points[name] = l, p

        ax.legend(loc='upper right', frameon=True, fancybox=True, shadow=True, fontsize=11)

        def init():
            for l, p in zip(lines.values(), points.values()):
                if view == '3D':
                    l.set_data([], []); l.set_3d_properties([])
                    p.set_data([], []); p.set_3d_properties([])
                else:
                    l.set_data([], []); p.set_data([], [])
            iter_text.set_text("")
            return list(lines.values()) + list(points.values()) + [iter_text]

        def animate(frame):
            total_iter = len(results['adagrad_traj']) - 1

            # 🔥 SMART SAMPLING: Map 200 frames to full 10K trajectory
            real_iter = int((frame / 199) * total_iter)  # Scale to full trajectory

            for name in colors:
                if name == 'Normal':
                    traj = np.array([[results['normal'][1], results['normal'][2]]])
                else:
                    traj_len = min(real_iter + 1, len(results[f'{name.lower()}_traj']))
                    traj = results[f'{name.lower()}_traj'][:traj_len, 1:3]

                if view == '3D':
                    z_vals = 0.5 * (traj[:,0]**2 + 4*traj[:,1]**2)
                    lines[name].set_data(traj[:, 0], traj[:, 1])
                    lines[name].set_3d_properties(z_vals)
                    points[name].set_data([traj[-1, 0]], [traj[-1, 1]])
                    points[name].set_3d_properties([z_vals[-1]])
                else:
                    lines[name].set_data(traj[:, 0], traj[:, 1])
                    points[name].set_data([traj[-1, 0]], [traj[-1, 1]])

            iter_text.set_text(f"Iter: {real_iter+1}/{total_iter}")
            return list(lines.values()) + list(points.values()) + [iter_text]

        # ✅ 200 FRAMES = Fast + Shows full 10K progress!
        max_frames = 200
        ani = animation.FuncAnimation(fig, animate, frames=max_frames,
                                     init_func=init, interval=100, blit=False, repeat=True)
        plt.close(fig)
        return ani

    # RUN
    total_iter = len(results['adagrad_traj']) - 1
    print(f"\n🌟 3D ANIMATION (200 frames showing {total_iter} iterations):")
    ani_3d = animate_convergence('3D')
    display(HTML(ani_3d.to_jshtml()))

    print(f"\n🌟 2D ANIMATION (200 frames showing {total_iter} iterations):")
    ani_2d = animate_convergence('2D')
    display(HTML(ani_2d.to_jshtml()))

    print(f"\n✅ PERFECT! 200 smooth frames map to your full {total_iter} iterations!")
    print("🟢 Green path shows COMPLETE 10K convergence!")


    # === UPLOAD TO GITHUB FROM COLAB ===
!git config --global user.email "1032251810@tcetmumbai.in"
!git config --global user.name "Aayush Kasurde"

# Clone your new repo
!git clone https://github.com/Aayush9305/LR-Optimization-Comparison-
%cd LR-Optimization-Comparison

# Create structure
!mkdir src notebooks data outputs

